# California Housing Project

In [ ]:
import requests
from pathlib import Path
import pandas as pd
import tarfile

from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.preprocessing import OneHotEncoder

## Load the Data

In [ ]:
def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        r = requests.get(url, stream=True)
        tarball_path.write_bytes(r.content)
        with tarfile.open(tarball_path, "r:gz") as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))


In [ ]:
df = load_housing_data()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# ocean_proximity is the only categorical feature in our df and we see that 5 categories exist
df["ocean_proximity"].value_counts()

In [ ]:
#summary of the numerical features
df.describe()

In [ ]:
# Histogram for each numerical features
import matplotlib.pyplot as plt

df.hist(bins=50, figsize=(12,8))
plt.show()

## Create a test set

In [ ]:
# We will do stratified sampling based on the income category

df["income_cat"] = pd.cut(df["median_income"], bins=[0., 1.5, 3.0, 4.5, 6, np.inf], labels=[1,2,3,4,5])

In [ ]:
df["income_cat"].value_counts().sort_index().plot.bar(rot=0, grid=True)
plt.xlabel("Income category")
plt.ylabel("Number of districts")
plt.show()

In [ ]:
# We will get a single split using the train_test_split() function with the stratify argment.
strat_train_set, strat_test_set = train_test_split(df, test_size=0.2, stratify=df["income_cat"], random_state=42)

In [ ]:
# We wont use income_cat again so we will drop it
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

## Exploratory Data Analysis

### Visualizing Geographical Data

In [ ]:
# This looks like California. We will set alpha = 0.2 to visualize the high-density areas

df.plot(kind="scatter", x="longitude", y="latitude", grid=True, alpha=0.2)
plt.show()

In [ ]:
df.plot(kind="scatter", x="longitude", y="latitude", grid=True, 
        s=df["population"] / 100, label="population",
        c="median_house_value", cmap="jet", colorbar=True,
        legend=True, sharex=False, figsize=(12,8))
plt.show()

In [ ]:
# red is expensive, blue is ceap, larger circles indicate areas with a larger population.

### Look for Correlations

In [ ]:
# Let's look  at how much each attribute correlates with the median house value

corr_matrix = df.select_dtypes(include=['number']).corr()

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

## Prepare the Data for Machine Learning Algorithms

In [ ]:
# drop() creates a copy of the data and does not affect strat_train_set
X_train = strat_train_set.drop("median_house_value", axis=1)
y_train = strat_train_set["median_house_value"].copy()

### Transformation Pipelines

In [ ]:
# Custom transformer: cluster similarity
class ClusterSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(n_clusters=self.n_clusters, random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self

    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)

    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

In [ ]:
# Making a complete Pipeline that:
# 1) Missing values in numerical features will be imputed by replacing them with the median. 
#    Categorical features missing values will be replaced by the most frequent category.

# 2) The categorical features will be one-hot encoded.

# 3) A few ratio features will be computed and added ( betrooms_ratio, rooms_per_house, people_per_house)
#    Hopefully these will better correlate with the median house value.

# 4) A few cluster similarity features will be added.
#    Hopefully these will be better than latitude and longitude.

# 5) Features with a long tail will be replaced by their logarithm, to achieve a distribution closer to Gaussian distribution.

# 6) All numerical features will be standardized (scaled)

In [ ]:
def column_ratio(X):
    return X[:, [0]] / X[:, [1]]

def ratio_name(function_transformer, feature_names_in):
    return ["ratio"]

def ratio_pipeline():
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler())

log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log1p, feature_names_out="one-to-one"),
    StandardScaler())

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore")
)

cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1, random_state=42)

default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"),
                                     StandardScaler())

preprocessing = ColumnTransformer([
    ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
    ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]),
    ("people_per_house", ratio_pipeline(), ["population", "households"]),
    ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population", "households", "median_income"]),
    ("geo", cluster_simil, ["latitude", "longitude"]),
    ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
],
remainder=default_num_pipeline)  #one column remaining: housing_median_age

In [ ]:
X_prepared = preprocessing.fit_transform(X_train)

## Select and Train a Model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lin_reg = make_pipeline(preprocessing, LinearRegression())
lin_reg.fit(X_train, y_train)

In [ ]:
y_pred = lin_reg.predict(X_train)


In [ ]:
from sklearn.metrics import root_mean_squared_error

In [ ]:
lin_rmse = root_mean_squared_error(y_train, y_pred)

In [ ]:
lin_rmse

In [ ]:
#we have an error of ~$69.000
# The model is underfitting the training data

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=42))
tree_reg.fit(X_train, y_train)

In [ ]:
y_pred = tree_reg.predict(X_train)
tree_rmse = root_mean_squared_error(y_train, y_pred)
tree_rmse

In [ ]:
#The model overfitted badly

### Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score

tree_rmse = -cross_val_score(tree_reg, X_train, y_train, scoring="neg_root_mean_squared_error", cv=10)

pd.Series(tree_rmse).describe()

#Now we see RMSE of 66868.

# We know thtat there is severe overfitting because the training error is low (0) and the validation error is high.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = make_pipeline(preprocessing, RandomForestRegressor(random_state=42))

forest_rmse = -cross_val_score(forest_reg, X_train, y_train, scoring="neg_root_mean_squared_error", cv=10)

pd.Series(forest_rmse).describe()

In [ ]:
forest_reg = make_pipeline(preprocessing, RandomForestRegressor(random_state=42))
forest_reg.fit(X_train, y_train)
y_pred = forest_reg.predict(X_train)
forest_rmse = root_mean_squared_error(y_train, y_pred)
forest_rmse

In [ ]:
# We see that RMSE on training set is 17474 and RMSE on validation set is 47038 meaning that still there is much overfitting 

## Fine-Tune 

### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("random_forest", RandomForestRegressor(random_state=42)),
])

param_grid = [
    {'preprocessing__geo__n_clusters':[5,8,10],
     'random_forest__max_features':[4,6,8]},
    {'preprocessing__geo__n_clusters':[10,15],
     'random_forest__max_features':[6,8,10]},
]

grid_search = GridSearchCV(full_pipeline, param_grid, cv=3,scoring='neg_root_mean_squared_error')

grid_search.fit(X_train, y_train)

In [ ]:
# best model has n_clusters=15 and max_features=6
grid_search.best_params_

In [ ]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res.head()

In [ ]:
# we get a score of 43589 which is better than 47038 we had before.

### Randomized Search using RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs={'preprocessing__geo__n_clusters': randint(low=3,high=50),
                'random_forest__max_features':randint(low=2, high=20)}

rnd_search = RandomizedSearchCV(full_pipeline, param_distributions=param_distribs, n_iter=10, cv=3,
                                scoring='neg_root_mean_squared_error', random_state=42)

rnd_search.fit(X_train, y_train)

## Final Model

In [ ]:
final_model = rnd_search.best_estimator_
feature_importances = final_model["random_forest"].feature_importances_
feature_importances.round(2)

In [ ]:
sorted(zip(feature_importances, final_model["preprocessing"].get_feature_names_out()), reverse=True)

In [ ]:
# We can use sklearn.feature_selection.SelectFromModel to automatically drop the least useful features

## Evaluate on the test set

In [ ]:
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

final_predictions = final_model.predict(X_test)



In [ ]:
final_rmse = root_mean_squared_error(y_test, final_predictions)
print(final_rmse)

In [ ]:
from scipy import stats

def rmse(squared_errors):
    return np.sqrt(np.mean(squared_errors))

confidence = 0.95
squared_errors = (final_predictions - y_test)**2
boot_result = stats.bootstrap([squared_errors], rmse, confidence_level=confidence, random_state=42)

rmse_lower, rmse_upper = boot_result.confidence_interval

## Launch

In [ ]:
import joblib

joblib.dump(final_model, "my_california_housing_model.pkl")